In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1377615/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.7)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_9_15,0.759870,0.091054,0.860195,0.446972,0.791156,0.393662,6.078128,0.831912,0.756603,0.794257,0.643045,0.627425,1.889894,0.637464,161.864526,334.575173,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
1,model_1_9_16,0.759839,0.090435,0.858165,0.441987,0.788672,0.393713,6.082265,0.843992,0.763422,0.803707,0.641434,0.627466,1.890010,0.637506,161.864265,334.574912,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
2,model_1_9_14,0.759822,0.091657,0.862191,0.451767,0.793580,0.393741,6.074091,0.820036,0.750042,0.785039,0.644728,0.627488,1.890073,0.637528,161.864124,334.574770,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
3,model_1_9_17,0.759727,0.089798,0.856101,0.436806,0.786125,0.393897,6.086527,0.856273,0.770511,0.813392,0.639929,0.627612,1.890425,0.637655,161.863331,334.573978,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
4,model_1_9_13,0.759695,0.092244,0.864156,0.456372,0.795945,0.393949,6.070172,0.808344,0.743743,0.776043,0.646500,0.627654,1.890544,0.637697,161.863065,334.573712,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,model_1_8_4,0.706160,0.104401,-0.286602,0.755759,0.440291,0.481712,5.988878,1.652601,0.716686,1.184644,0.821724,0.694055,2.088935,0.705160,161.460819,334.171465,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
83,model_1_8_3,0.703626,0.104654,-0.298737,0.756937,0.437425,0.485867,5.987180,1.668189,0.713229,1.190709,0.825966,0.697042,2.098329,0.708195,161.443640,334.154287,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
84,model_1_8_2,0.700973,0.104924,-0.311446,0.758052,0.434342,0.490216,5.985375,1.684513,0.709957,1.197235,0.830258,0.700154,2.108159,0.711357,161.425820,334.136466,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
85,model_1_8_1,0.698200,0.105209,-0.324729,0.759092,0.431032,0.494762,5.983472,1.701575,0.706906,1.204240,0.834600,0.703393,2.118435,0.714648,161.407358,334.118005,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
